In [1]:
!pip install pandas
!pip install numpy

DEPRECATION: o365 2.0.16 has a non-standard dependency specifier tzlocal<3.*,>=1.5.0. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of o365 or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


DEPRECATION: o365 2.0.16 has a non-standard dependency specifier tzlocal<3.*,>=1.5.0. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of o365 or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import os

# SNOMED-CT International Version Used
version = '20240301'

In [3]:
# Load the SNOMED CT concept, description, and language refset file
concept = pd.read_table('sct2_Concept_Snapshot_INT_{version}.txt'.format(version = version), sep='\t')
description = pd.read_table('sct2_Description_Snapshot-en_INT_{version}.txt'.format(version = version), sep='\t')
language_ref_set = pd.read_table(os.path.join('..', 'RefSet', 'Language' , 'der2_cRefset_LanguageSnapshot-en_INT_{version}.txt'.format(version = version)), sep='\t')

# Read the SNOMED CT >< Orphanet Map from Snapshot\Refset\Map\der2_sRefset_OrphanetSimpleMapSnapshot_INT_20230731.txt - The latest version at March 2024 is 20230731 map
orphanet_map = pd.read_table('der2_sRefset_OrphanetSimpleMapSnapshot_INT_20230731.txt', sep='\t')

### Combine Concept, Description, and Language Refset file from SNOMED CT International RF2 Release

In [4]:
# Explore how many row in concept, description, and language refset file
print('concept :', [x for x in concept.columns], len(concept), 'rows')
print('description :', [x for x in description.columns], len(description), 'rows')
print('language_ref_set :', [x for x in language_ref_set.columns], len(language_ref_set), 'rows')
print("\n")

# Print Statistic of the concept and description
print("Total concept : ", len(concept), " | Total active : ", len(concept[concept['active'] == 1]), " | Delta : ", len(concept) - len(concept[concept['active'] == 1]))
print("Total description : ", len(description), " | Total active : ", len(description[description['active'] == 1]), " | Delta : ", len(description) - len(description[description['active'] == 1]))

## Subset and print active concept number
active_concept = concept[concept['active'] == 1]
concept_list = active_concept['id'].tolist()
print("Active Concept : ", len(concept_list))
print("\n")

## Active description contained in Active concept
filtered_description = description[description['conceptId'].isin(concept_list)]
active_description = filtered_description[filtered_description['active'] == 1]
fsn_description = active_description[active_description['typeId'] == 900000000000003001]
synonym_description = active_description[active_description['typeId'] == 900000000000013009]

print('Active Description : ', len(active_description))
print('FSN Description : ', len(fsn_description))
print('Synonym Description : ', len(synonym_description))
print("\n")

## active language refset
filtered_language_ref_set = language_ref_set[language_ref_set['referencedComponentId'].isin(synonym_description['id'].tolist())]
active_language_ref_set = filtered_language_ref_set[filtered_language_ref_set['active'] == 1]

## Value Counts
## 900000000000548007 = Preferred, 900000000000549004 = Acceptable
# active_language_ref_set['acceptabilityId'].value_counts()

active_language_ref_set['preferability'] = active_language_ref_set['acceptabilityId'].apply(lambda x: 'Preferred' if x==900000000000548007 else 'Acceptable')

trimmed_language_ref_set = active_language_ref_set[['referencedComponentId', 'preferability']].drop_duplicates()
print('Active Language Refset from active concept & active description : ', len(active_language_ref_set))
print('Trimmed / drop duplicates : ', trimmed_language_ref_set['preferability'].count())
print("\n")

## active description
## 900000000000003001 = FSN
active_description['type'] = active_description['typeId'].apply(lambda x: 'FSN' if x==900000000000003001 else 'Synonym')

trimmed_active_description = active_description[['id', 'conceptId', 'term', 'type']].drop_duplicates()
print('Active Description from active concept : ', len(active_description))
print('Trimmed / drop duplicates : ', len(trimmed_active_description))

concept : ['id', 'effectiveTime', 'active', 'moduleId', 'definitionStatusId'] 511088 rows
description : ['id', 'effectiveTime', 'active', 'moduleId', 'conceptId', 'languageCode', 'typeId', 'term', 'caseSignificanceId'] 1622713 rows
language_ref_set : ['id', 'effectiveTime', 'active', 'moduleId', 'refsetId', 'referencedComponentId', 'acceptabilityId'] 3173754 rows


Total concept :  511088  | Total active :  366651  | Delta :  144437
Total description :  1622713  | Total active :  1328439  | Delta :  294274
Active Concept :  366651


Active Description :  969140
FSN Description :  366632
Synonym Description :  602508




<ipython-input-4-f1816cea5952>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  active_language_ref_set['preferability'] = active_language_ref_set['acceptabilityId'].apply(lambda x: 'Preferred' if x==900000000000548007 else 'Acceptable')


Active Language Refset from active concept & active description :  1139646
Trimmed / drop duplicates :  609376




<ipython-input-4-f1816cea5952>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  active_description['type'] = active_description['typeId'].apply(lambda x: 'FSN' if x==900000000000003001 else 'Synonym')


Active Description from active concept :  969140
Trimmed / drop duplicates :  969140


In [5]:
## Create combined DataFrame
combined = pd.merge(trimmed_active_description, trimmed_language_ref_set, left_on='id', right_on='referencedComponentId', how='left')
combined = combined.drop(['referencedComponentId', 'id'], axis=1).sort_values('conceptId').drop_duplicates()

## Combine type and preferability
combined['type_preferability'] = combined['type'] + ' - ' + combined['preferability']
combined.loc[combined['type_preferability'].isnull(), 'type_preferability'] = combined['type']

## Get Semantic Tag from FSN and save it in 'parent' column from FSN
fsn_description['parent'] = fsn_description['term'].apply(lambda x: x.split('(')[1].replace(')', ''))
raw_table_parent = fsn_description[['conceptId', 'parent']].drop_duplicates()

## Left join to get full table
combined = pd.merge(combined, raw_table_parent, left_on='conceptId', right_on='conceptId', how='left')
# combined.head()

## Finishing touch
final_df = pd.DataFrame(columns = ['hierarchy', 'system', 'code', 'display', 'type', 'version'] )
final_df[['hierarchy', 'code', 'display', 'type']] = combined[['parent', 'conceptId', 'term', 'type_preferability']]
final_df['system'] = 'http://snomed.info/sct'
final_df['version'] = version
final_df.head()

### Convert to final_df to csv (Full SNOMED CT List)
# final_df.to_csv('Full Table Ready SNOMED-CT.csv', index=False, sep=';')

### Apply filter for FSN and Synonym - Preferred Description
# filter = ['FSN', 'Synonym - Preferred']
# final_df = final_df[final_df['type'].isin(filter)]
# final_df.to_csv('Filtered Table Ready SNOMED-CT.csv', index=False, sep=';')

<ipython-input-5-959290f13778>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fsn_description['parent'] = fsn_description['term'].apply(lambda x: x.split('(')[1].replace(')', ''))


,hierarchy,system,code,display,type,version
0,organism,http://snomed.info/sct,101009,Quilonia ethiopica (organism),FSN,20240301
1,organism,http://snomed.info/sct,101009,Quilonia ethiopica,Synonym - Preferred,20240301
2,substance,http://snomed.info/sct,102002,Hemoglobin Okaloosa,Synonym - Preferred,20240301
3,substance,http://snomed.info/sct,102002,"Hb 48(CD7), Leu-arg",Synonym - Acceptable,20240301
4,substance,http://snomed.info/sct,102002,Haemoglobin Okaloosa,Synonym - Preferred,20240301


In [6]:
## Reduce repetition of similar row - Tile FSN and Synonym horizontally
fsn = final_df[final_df['type'] == 'FSN']
preferred = final_df[final_df['type'] == 'Synonym - Preferred'].drop_duplicates(subset=['code'])
acceptable = final_df[final_df['type'] == 'Synonym - Acceptable'].drop_duplicates(subset=['code'])

## Create FSN Column Subset
cut_fsn = pd.DataFrame()
cut_fsn[['hierarchy', 'system', 'code', 'fsn', 'version']] = fsn[['hierarchy', 'system', 'code', 'display', 'version']]

## Create Preferred Synonym Column Subset
cut_preferred = pd.DataFrame()
cut_preferred[['code', 'preferred']] = preferred[['code', 'display']]

## Create Acceptable Synonym Column Subset
cut_acceptable = pd.DataFrame()
cut_acceptable[['code', 'acceptable']] = acceptable[['code', 'display']]

## Statistic 
print('FSN number :', len(cut_fsn))
print('Preferred Syn number : ', len(cut_preferred))
print('Acceptable Syn number : ', len(cut_acceptable))
print('\n')

FSN number : 366632
Preferred Syn number :  366621
Acceptable Syn number :  138687




In [7]:
# Merge column subset
trimmed_row = pd.merge(cut_fsn, cut_preferred, left_on='code', right_on='code', how='left')
trimmed_row = pd.merge(trimmed_row, cut_acceptable, left_on='code', right_on='code', how='left')
# trimmed_row.to_csv('Concise SNOMED Table.tsv', index=False, sep='\t')

### Merge with Orphanet Mapping

In [8]:
# Left join from referencedComponentId
combined_orphanet = pd.merge(trimmed_row, orphanet_map, left_on='code', right_on='referencedComponentId', how='left')

# combined_orphanet with non null mapTarget
combined_orphanet = combined_orphanet[combined_orphanet['mapTarget'].notnull()]
combined_orphanet['orphanet_map'] = combined_orphanet['mapTarget'].apply(lambda x: str(x).replace('.0', ''))

# Drop all column from the der2_sRefset_OrphanetSimpleMapSnapshot_INT_20230731.txt except mapTarget
combined_orphanet = combined_orphanet.drop(['id', 'effectiveTime', 'active', 'moduleId', 'refsetId', 'referencedComponentId', 'mapTarget'], axis=1)
# combined_orphanet.head()

# Add prefix ORPHA and SNOMED to code and orphanet_map
combined_orphanet['code'] = 'SNOMEDCT:' + combined_orphanet['code'].astype(str)
combined_orphanet['orphanet_map'] = 'ORPHA:' + combined_orphanet['orphanet_map'].astype(str)

# Save to TSV
combined_orphanet.to_csv(os.path.join('subset', 'snomed2orpha_subset.tsv'), index=False, sep='\t')